<H1>Applied Data Science Capstone Part 1</H1>

<H3>First, I install the needed packages</H3>

In [49]:
#Installing needed packages

import numpy as np

import pandas as pd

import requests

import urllib.request

!pip install beautifulsoup4

from bs4 import BeautifulSoup

<H3>Then, I use BeautifulSoup to scrape the wikipedia page</H3>

In [50]:
#Loading Wikipedia page
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    
#save the HTML into variable
page = urllib.request.urlopen(wiki)

#srapping data into variable "soup"
soup = BeautifulSoup(page)

In [51]:
#Create Dataframe with scrapped data

wiki_table = soup.find('table', class_ = 'wikitable sortable')

A=[]
B=[]
C=[]

for row in wiki_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

CA_df=pd.DataFrame(A,columns=['PostalCode'])
CA_df['Borough']=B
CA_df['Neighborhood']=C

<H3>Rows with Not assigned Boroughs are useless, so I drop them</H3>

In [54]:
#Dropping rows with 'Not assigned' in column 'Borough'

indexNames = CA_df[ CA_df['Borough'] == 'Not assigned' ].index

CA_df.drop(indexNames , inplace=True)

CA_df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


<H3>I reassign 'Not assigned' Neighborhood names with their respective Borough names</H3>

In [55]:
#Reassigning 'Not assigned' Neighborhood names with 'Borough' values

CA_df.loc[df['Neighborhood'].str.contains('Not'), 'Neighborhood'] = CA_df['Borough']

CA_df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


<H3>I combine Neighborhoods by Postal Codes and Boroughs</H3>

In [56]:
#Combine Neighborhoods if postal code is the same

CA_df=CA_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

#Last round of cleanup
CA_df.replace('\n','',regex=True,inplace=True)
CA_df.head(20)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<H3>Finally, I see the result of the work by using the .shape method</H3>

In [58]:
CA_df.shape

(103, 3)